## Step 1: Select from the Examples

In [ ]:
from __future__ import print_function

from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.structure import StructureData
from aiida.orm.calculation import Calculation

from aiida.tools.dbimporters.plugins.cod import CodDbImporter

import ase.io
import numpy as np
import ipywidgets as ipw
from base64 import b64decode
from IPython.display import display, clear_output, Image, HTML
from fileupload import FileUploadWidget

import nglview

In [ ]:
atoms = None
structures = [("select structure",{"status":False})]

layout = ipw.Layout(width="400px")
style = {"description_width":"150px"}

viewer = nglview.NGLWidget()
clear_output()

In [ ]:
def get_example_structure(key):
    from aiida.orm import DataFactory
    from ase.lattice.spacegroup import crystal
    StructureData = DataFactory('structure')
    
    if key == 'diamond':
        # This is the lattice constant in angstrom
        alat = 3.56
        diamond_ase = crystal('C', [(0,0,0)], spacegroup=227,
                          cellpar=[alat, alat, alat, 90, 90, 90],primitive_cell=True)
        s = StructureData(ase=diamond_ase)
        return s
    elif key == 'al':
        # This is the lattice constant in angstrom
        alat = 4.05
        Al_ase = crystal('Al', [(0,0,0)], spacegroup=225,
                          cellpar=[alat, alat, alat, 90, 90, 90],primitive_cell=False)
        s = StructureData(ase=Al_ase)
        return s
    elif key == 'gaas':
        # This is the lattice constant in angstrom
        alat = 5.75
        GaAs_ase = crystal('GaAs', [(0,0,0),(0.25,0.25,0.25)], spacegroup=216,
                          cellpar=[alat, alat, alat, 90, 90, 90],primitive_cell=True)
        s = StructureData(ase=GaAs_ase)
        return s
    elif key == 'co':
        # These are the lattice constants in angstrom
        a = 2.5
        c = 4.07
        Co_ase = crystal('Co', [(1./3,2./3,0.25)], spacegroup=194,
                          cellpar=[a, a, c, 90, 90, 120],primitive_cell=True)
        s = StructureData(ase=Co_ase)
        return s
    else:
        raise ValueError("Unknown or unsupported example structure '{}'".format(key))

def get_structure():
    global structure
    if in_use_example_structure.value:
        structure_key = in_example_structure.value
        return get_example_structure(structure_key)
    else:
        if structure is None:
            raise ValueError("You did not upload a structure. Either upload a structure or choose an example.")
        else:
            return structure
structures = [
    ("select structure",{'status':False}),
    ('Diamond', {'structure':get_example_structure('diamond')}),
    ('Al', {'structure':get_example_structure('al')}),
    ('GaAs', {'structure': get_example_structure('gaas')}),
    ('CO', {'structure':get_example_structure('co')}),
]



In [ ]:
def refresh_structure_view():
    global viewer, atoms
    if hasattr(viewer, "component_0"):
        viewer.clear_representations()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_unitcell()
        cid = viewer.component_0.id
        viewer.remove_component(cid)

    viewer.add_component(nglview.ASEStructure(atoms.get_ase())) # adds ball+stick
    viewer.add_unitcell()
    viewer.center()

In [ ]:
def on_change(c):
    global atoms
    indx = c['owner'].index
    new_element = c['new']
    atoms = new_element['structure']
    formula = atoms.get_ase().get_chemical_formula()
    refresh_structure_view()


    
drop_structure = ipw.Dropdown(description="", options=structures, style=style, layout=layout )
drop_structure.observe(on_change, names='value')
#link = ipw.HTML("Link to the web-page will appear here")
display(drop_structure, ipw.VBox([viewer]))

## Step 2: Store in AiiDA Database

In [ ]:
def on_click_store(b):
    global atoms
    with store_out:
        clear_output()
        if atoms is None:
            print ("Specify a structure first!")
            return
        s = StructureData(ase=atoms.get_ase())
        # ensure that tags got correctly translated into kinds 
        s.description = inp_descr.value
        s.store()
        print("Stored in AiiDA: "+repr(s))

inp_descr = ipw.Text(placeholder="Description (optional)")   
btn_store = ipw.Button(description='Store in AiiDA')
btn_store.on_click(on_click_store)

store_out = ipw.Output()
display(ipw.HBox([btn_store, inp_descr]), store_out)